# 

In [1]:
import pandas as pd
from bioinfokit.analys import Fasta
import os

In [2]:
ont = ["K10B", "K10f", "LK_1.3.fasta"]
genome = ""
te = "Alex1_Alex3.fasta"

In [3]:
for file in ont:
    os.system(f"seqtk seq -a ont/{file}.fastq.gz > ont/{file}.fasta")
#!seqtk seq -a ont/K10B.fastq.gz > ont/K10B.fasta
for file in ont:
    os.system(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
#!makeblastdb -in ont/K10B.fasta -dbtype nucl -out ont/K10B
!mkdir -p blast
for file in ont:
    os.system(f"blastn -num_threads 20 -outfmt 6 -query te/{te} -db ont/{file} -out blast/TE_{file}.bl -dust no -perc_identity 0.9")
#!blastn -num_threads 20 -outfmt 6 -query te/Alex1_Alex3.fasta -db ont/K10B -out blast/Alex_K10B.bl -dust no -perc_identity 0.9

[E::stk_seq] failed to open the input file/stream.




Building a new DB, current time: 03/25/2024 13:14:32
New DB name:   /home/kiki/data/ont/K10B
New DB title:  ont/K10B.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6939343 sequences in 334.228 seconds.




Building a new DB, current time: 03/25/2024 13:20:08
New DB name:   /home/kiki/data/ont/K10f
New DB title:  ont/K10f.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 4775182 sequences in 232.108 seconds.




Building a new DB, current time: 03/25/2024 13:24:01
New DB name:   /home/kiki/data/ont/LK_1.3.fasta
New DB title:  ont/LK_1.3.fasta.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B


BLAST options error: File ont/LK_1.3.fasta.fasta is empty
FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 111: 1-47
FASTA-Reader: Ignoring invalid residues at position(s): On line 114: 45-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 115: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 116: 1-17
FASTA-Reader: Ignoring invalid residues at position(s): On line 142: 15-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 143: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 144: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 145: 1-42
FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at positio

In [3]:
headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']

In [4]:
headers_dict = {i[0]:i[1] for i in enumerate(headers)}

In [5]:
def read_fasta(filename):
    return pd.read_csv(filename, sep="\t", header=None).rename(headers_dict, axis=1)
def save_fasta(df, filename):
    df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)

In [6]:
data = read_fasta("blast/TE_K10B.bl")

In [7]:
alex1 = data[data.qseqid.str.contains('Alex1')]
plus_alex1 = alex1[alex1.sstart < alex1.send]
minus_alex1 = alex1[alex1.sstart > alex1.send]

In [8]:
alex3 = data[data.qseqid.str.contains('Alex3')]
plus_alex3 = alex3[alex3.sstart < alex3.send]
minus_alex3 = alex3[alex3.sstart > alex3.send]

In [9]:
len(alex1) + len(alex3)

1732

In [10]:
plus_alex1.head(2)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,expect value,bitscore
902,Alex1_chr1:18795335-18799058,2c40afb0-7c90-4ddf-847c-a587e291d4a9,94.235,2810,67,76,949,3724,2130,4878,0.0,4204.0
903,Alex1_chr1:18795335-18799058,2c40afb0-7c90-4ddf-847c-a587e291d4a9,95.327,535,7,15,421,949,612,1134,0.0,833.0


In [11]:
bed_df = plus_alex1[['sseqid', 'sstart', 'send', 'qseqid']].sort_values(by=['sseqid', 'sstart', 'send'])


In [12]:
bed_df.head()

,sseqid,sstart,send,qseqid
1083,0033bcbe-f78a-48ec-9149-88b241b8febb,3445,3714,Alex1_chr1:18795335-18799058
1082,0033bcbe-f78a-48ec-9149-88b241b8febb,3445,4367,Alex1_chr1:18795335-18799058
1081,0033bcbe-f78a-48ec-9149-88b241b8febb,5341,8025,Alex1_chr1:18795335-18799058
1084,0033bcbe-f78a-48ec-9149-88b241b8febb,7757,8025,Alex1_chr1:18795335-18799058
1492,012fd23f-1632-4f4a-8428-eb3c56914692,36,256,Alex1_chr1:18795335-18799058


In [13]:
save_fasta(bed_df, 'raw_first_blast.bed')

In [14]:
!bedtools merge -i raw_first_blast.bed -d 1500 -c 4 -o collapse  > first_blast.bed.merged

In [15]:
merged = pd.read_csv(
    "first_blast.bed.merged", sep="\t", header=None).rename(
    {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
)

In [16]:
len(merged)

295

In [17]:
merged.head(2)

,sseqid,sstart,send,qseqid
0,0033bcbe-f78a-48ec-9149-88b241b8febb,3445,8025,"Alex1_chr1:18795335-18799058,Alex1_chr1:187953..."
1,012fd23f-1632-4f4a-8428-eb3c56914692,36,433,"Alex1_chr1:18795335-18799058,Alex1_chr1:187953..."


In [18]:
filtered_first_bed = merged[(merged.send - merged.sstart) > 4000]

In [19]:
pd.options.mode.copy_on_write = True
filtered_first_bed['end'] = filtered_first_bed['send'] + 4000
filtered_first_bed['start'] = filtered_first_bed['sstart'] - 4000
save_fasta(filtered_first_bed[['sseqid', 'sstart', 'send']], 'normal.txt')


In [21]:
save_fasta(filtered_first_bed[['sseqid']], 'seq_list.txt')

In [25]:
!seqtk subseq ont/K10B.fasta seq_list.txt > normal.fasta

In [26]:
!bedtools maskfasta -fi normal.fasta -bed normal.txt -fo Alex_K10B.masked.fasta
!seqkit fx2tab --length --name  Alex_K10B.masked.fasta > max_lenght.txt
max_len = pd.read_csv(
    'max_lenght.txt', sep="\t", header=None).rename(
    {0:'sseqid', 1:'max_len'}, axis=1).sort_values(by=['sseqid'])

In [27]:
#sseqid bo query to po prostu te w tym przypadku
save_fasta(filtered_first_bed[['sseqid', 'start', 'end']], 'cutted.txt')

mask_min = filtered_first_bed.start < 0
filtered_first_bed.loc[mask_min, 'start'] = 0
filtered_first_bed = pd.merge(filtered_first_bed, max_len, how='left', on="sseqid")
mask_max = filtered_first_bed.end > filtered_first_bed.max_len
filtered_first_bed.loc[mask_max, 'end'] = filtered_first_bed.max_len
save_fasta(filtered_first_bed[['sseqid', 'start', 'end']], 'cutted2.txt')


In [28]:
!seqtk subseq Alex_K10B.masked.fasta cutted2.txt > cutted_ont.fasta

In [135]:
#!mv output.fasta new2.fasta

# Genome masking

In [33]:
!makeblastdb -in genome/DCARv3.4.fa  -dbtype nucl -out genome/DCARv3.4



Building a new DB, current time: 03/25/2024 19:37:29
New DB name:   /home/kiki/data/genome/DCARv3.4
New DB title:  genome/DCARv3.4.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 7.92409 seconds.




In [34]:
# Blast Alex to genome
!blastn -num_threads 20 -outfmt 6 -query te/Alex1_Alex3.fasta -db genome/DCARv3.4 -out TE_genome.bl -perc_identity 0.8 -max_target_seqs 4

FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 111: 1-47
FASTA-Reader: Ignoring invalid residues at position(s): On line 114: 45-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 115: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 116: 1-17
FASTA-Reader: Ignoring invalid residues at position(s): On line 142: 15-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 143: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 144: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 145: 1-42


In [35]:
tex = read_fasta("TE_genome.bl")

In [37]:
#save_fasta(tex[["sseqid", "sstart", "send"]], "to_mask_genome.txt")
tex["start"] = tex['sstart'].where(tex['sstart'] <= tex["send"], other=tex['send'])
tex["stop"] = tex['sstart'].where(tex['sstart'] > tex["send"], other=tex['send'])
save_fasta(tex[["sseqid", "start", "stop"]], "to_mask_genome.txt")
!bedtools maskfasta -fi genome/DCARv3.4.fa -bed to_mask_genome.txt -fo TE_genome.masked.fasta

In [42]:
#okno = [record['sstart'], record['send'] if record['sstart'] >= record['send'] else None for record in tex]
#df['Conditioned_Numbers'] = df['Numbers'].where(df['Numbers'] > threshold, other='Below Threshold')
tex["start"] = tex['sstart'].where(tex['sstart'] <= tex["send"], other=tex['send'])
tex["stop"] = tex['sstart'].where(tex['sstart'] > tex["send"], other=tex['send'])
plus_tex = tex[texl.sstart < texl.send]
minus_tex = texl[alex3.sstart > texl.send]
save_fasta(tex[["sseqid", "start", "stop"]], "to_mask_genome.txt")

In [39]:
tex.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,expect value,bitscore,start,stop
0,Alex3,DCARv3_Chr6,99.793,2893,6,0,1,2893,33630943,33628051,0.000000e+00,5310.0,33628051,33630943
1,Alex3,DCARv3_Chr6,99.536,2153,10,0,2892,5044,33628329,33626177,0.000000e+00,3923.0,33626177,33628329
2,Alex3,DCARv3_Chr6,99.571,233,1,0,4812,5044,33630944,33630712,1.420000e-116,425.0,33630712,33630944
3,Alex3,DCARv3_Chr6,99.569,232,1,0,1,232,33626408,33626177,5.090000e-116,424.0,33626177,33626408
4,Alex3,DCARv3_Chr3,99.793,2893,6,0,1,2893,3170546,3167654,0.000000e+00,5310.0,3167654,3170546


# Second part of analysis

In [43]:
!blastn -num_threads 20 -outfmt 6 -query cutted_ont.fasta -db genome/DCARv3.4 -out Alex_K10B_genome.bl -perc_identity 0.8 -max_target_seqs 4

In [44]:
result = pd.read_csv("Alex_K10B_genome.bl", sep="\t", header=None).rename(headers_dict, axis=1)

In [53]:
plus_result = result[result.sstart <= result.send]
minus_result = result[result.sstart > result.send]
len(plus_result)

1254

In [54]:
len(result.send)

2489

In [58]:
def _create_part_raport(df):
    df = df.sort_values(by=['sseqid', 'sstart', 'send'])
    start_te = df.send[:-1]
    start_te = start_te.reset_index(drop=True)
    end_te = df.sstart[1:]
    end_te = end_te.reset_index(drop=True)
    return pd.DataFrame({'qseqid':df.qseqid.values[0], 'sseqid':df.sseqid.values[0], 'start_te': start_te, 'end_te': end_te})

def create_report(df):
    series_list = []
    for qseqid in df.qseqid.unique():
        okno = df[df.qseqid == qseqid]
        for sseqid in okno.sseqid.unique():
            okno2 = okno[(okno.sseqid == sseqid)]
            series_list.append(_create_part_raport(okno2))
    return pd.concat(series_list)


In [60]:
plus = create_report(plus_result)
minus = create_report(minus_result)

plus.to_csv("plus_report", sep='\t', index=False, encoding='utf-8', header=False)
minus.to_csv("minus_report", sep='\t', index=False, encoding='utf-8', header=False)

In [62]:
plus['length'] = plus.end_te - plus.start_te
minus['length'] = minus.end_te - minus.start_te

In [64]:
plus.to_csv("plus_report2", sep='\t', index=False, encoding='utf-8', header=True) # i add header
minus.to_csv("minus_report2", sep='\t', index=False, encoding='utf-8', header=True) # i add header

In [65]:
finish_plus = plus[(plus.length < 8000) & (plus.length > 4000)]
finish_minus = minus[(minus.length < 8000) & (minus.length > 4000)]

In [66]:
save_fasta(finish_plus, 'finish_plus')
save_fasta(finish_minus, 'finish_minus')


In [69]:
finish_plus["strain"] = "+"
finish_minus["strain"] = "-"
ni =  pd.concat([finish_plus, finish_minus]).sort_values(by=['sseqid', 'start_te'])

In [70]:
ni.head(30)

,qseqid,sseqid,start_te,end_te,length,strain
0,9676ad26-b838-4bf4-b6f3-803f3cb06ff0:8506-20811,DCARv3_Chr1,2879944,2884063,4119,-
0,2c40afb0-7c90-4ddf-847c-a587e291d4a9:1-9021,DCARv3_Chr1,2879983,2884668,4685,-
1,9676ad26-b838-4bf4-b6f3-803f3cb06ff0:8506-20811,DCARv3_Chr1,2883315,2888128,4813,-
1,69ef9f65-ed6a-4216-b76a-68c01c61dbe1:26138-36260,DCARv3_Chr1,7417810,7423816,6006,+
0,255e5e59-5e10-49f6-9460-36d8ab9bfc97:2183-11806,DCARv3_Chr1,24857327,24862059,4732,-
1,9ebca745-06aa-4187-9303-3fc73853bf74:1-10652,DCARv3_Chr1,32967973,32973561,5588,-
2,9ebca745-06aa-4187-9303-3fc73853bf74:1-10652,DCARv3_Chr1,32969507,32975637,6130,-
5,98c0c670-f831-449a-a2d7-6b50aad7d341:308-12808,DCARv3_Chr1,45816544,45822054,5510,-
6,98c0c670-f831-449a-a2d7-6b50aad7d341:308-12808,DCARv3_Chr1,45818829,45822962,4133,-
1,bed145d3-72e9-4002-ba69-3a2a28356019:1-9105,DCARv3_Chr1,49093136,49097161,4025,-
